In [1]:
import pandas as pd
import numpy as np
from util import *

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")

In [24]:
from tqdm.auto import tqdm
def build_data_set(data_set): 
    frames = []
    for tier in [tier1, tier2, tier3, tier4]:
        for label in tqdm(set(tier) & set(all_tiers_100)):
            nice_label = tier_translations.get(label)
            if nice_label is None:
                continue
            positives = data_set[data_set[label]].abstract.to_frame("positive")
            positives['fk'] = 0
            positives['label'] = f"This is labelled: {nice_label}."
            negatives = data_set[~data_set[label]].abstract.to_frame("negative")
            negatives['fk'] = 0
            triplets = pd.merge(positives, negatives, how="outer", on="fk")
            frames.append(triplets[['label', 'positive', 'negative']])
    return pd.concat(frames)

In [25]:
training_examples = build_data_set(training_set)

In [26]:
testing_examples= build_data_set(testing_set)

In [27]:
training_examples.to_parquet("training_triplets.parquet")

In [28]:
testing_examples.to_parquet("testing_triplets.parquet")

In [29]:
training_examples.shape

(3701622, 3)

In [30]:
testing_examples.shape

(228012, 3)